# <center> Converting .txt files into .pkl files <center>

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import mplhep as hep
from scipy.optimize import curve_fit
import pandas as pd
import hist
from hist import Hist

plt.style.use(hep.style.ROOT)

### User Defined

In [ ]:
with open('Run4_list.txt') as f:
    lines = f.read().split('\n')
tags = None
channels =[]
timeStamps = []
din={}
for line in lines:
    if line[:2]=="//" or len(line)==0:
        continue
    #first line after comments
    if tags is None:
        tags = line.split()
        tags=tags[-3:]
        continue;
    split = line.split()
    if(len(split) == 6):
        timeStamps.append(float(split[0]))
    channel, LG, HG = split[-3:]
    if channel not in channels:
        din[f"Ch_{channel}_LG"] = []
        din[f"Ch_{channel}_HG"] = []
        channels.append(channel)
    din[f"Ch_{channel}_LG"].append(float(LG))
    din[f"Ch_{channel}_HG"].append(float(HG))

din[f"TimeStamps"] = []
din[f"TimeStamps"] = timeStamps

#This says ptrig, but could be used for either ptrig or cosmic data. Just label which is which
ptrigDF = pd.DataFrame(din)
print(f"Done! Total events: {len(ptrigDF)}")
#ptrigDF.to_pickle("./COSMIC_LG50_HG50_4k_R26.pkl") 


In [ ]:
print(ptrigDF)
#ptrigDF.to_pickle('./COSMIC_LG50_HG50_4k_R3.pkl')

Plotting raw data for inspection

In [ ]:
#plot random trigger spectra for each low gain channel. Fit to gaussian and record pedestals for cuts later

fig, axs=plt.subplots(4, 10, figsize=(0.5*len(ptrigDF.columns),15),sharey=True,sharex=True)
plt.rcParams['figure.facecolor']='white'
plt.rcParams['savefig.facecolor']='white'
plt.rcParams['savefig.bbox']='tight'

mips = []
mipStds = []

for i in range(20):
    plt.sca(axs[i%4][i//4])

    h = hist.Hist(hist.axis.Regular(50, 0, 700, name="Ptrig"))
    
    cosmicCh = np.array(getattr(ptrigDF,"Ch_{}_LG".format("{:02d}".format(i))))
  
    h.fill(cosmicCh)
    #h.plot(color='tab:blue',label=f"Channel {i} Cosmic Data")  
    
    data = h.values()[()]
    bins = h.axes[0].edges[:-1]
    
    plt.errorbar(bins,data,yerr=np.sqrt(data),fmt='o',ecolor='tab:blue', color='tab:blue', capsize=0, elinewidth=1, markeredgewidth=0,label=f"Channel {i} Cosmic Data")

    
    # Find the bin number with the maximum value
    max_bin_number = np.argmax(h.values())

    # Get the axis from the histogram
    axis = h.axes[0]

    # Find the x-coordinate corresponding to the maximum bin
    x_coordinate = axis.edges[max_bin_number]
    #mips.append(x_coordinate)
    
    
    plt.legend(loc='upper right', fontsize=20)    
    
    # Set the title and axis labels for the subplot   
    if (i+1)%4 == 0:
        plt.xlabel('Signal Amplitude [ADC]')
    if i <= 3:
        plt.ylabel('Count')
    

# Adjust the spacing between subplots
plt.ylim(0,300)
plt.tight_layout(pad=0.0)
plt.subplots_adjust(hspace=0.0, wspace=0.0)